### 对用户行为数据进行预处理

    这篇 notebook 的主要作用是根据用户的历史行为信息进行用户画像,主要是挖掘用户的兴趣信息，然后分开为训练集和测试集输入到树模型中进行；
    具体方式为: 
      根据前7天的历史行为数据，对用户和物品进行进一步地描述和挖掘（也可以之后尝试不是7天的历史行为数据，而是根据所有历史行为数据进行挖掘）
      然后将挖掘到的信息，作为第8天的特征，将第八天的数据输入到树模型中进行训练；
      数据集一共包含了14天的数据，于是可以分别划分为8份，分别是：
      1. 挖掘 1 到 7 天的信息，作为第 8 天数据的特征；
      2. 挖掘 2 到 8 天的信息，作为第 9 天数据的特征；
      3. 挖掘 3 到 9 天的信息，作为第 10 天数据的特征；
      4. 挖掘 4 到 10 天的信息，作为第 11 天数据的特征；
      5. 挖掘 5 到 11 天的信息，作为第 12 天数据的特征；
      6. 挖掘 6 到 12 天的信息，作为第 13 天数据的特征；
      7. 挖掘 7 到 13 天的信息，作为第 14 天数据的特征；
      8. 挖掘 8 到 14 天的信息，作为第 15 天数据的特征；
      在线下训练阶段，选在第 8 天到第 13 天的数据作为训练集，而第 14 天的数据做验证；
      当需要test数据进行预测时候，则直接输入第 8 天到第 14 天的数据进行训练，而test是第 15 天的数据；

    需要挖掘的历史行为数据包括以下几个部分：
    1.用户兴趣挖掘，根据用户过去7天的行为数据，挖掘用户对于feed的偏好信息，然后将这些偏好信息或者直接输入模型，或者和对应的feed特征进行一些交叉；
      当前视频的特征包括：authorid、videoplaysecond、bgm_song_id、bgm_singer_id、author_n_feeds、top5_keywords、top5_keywords_weights、tag_list、tag_weights、feed_embedding
      对于这些特征，可以依次挖掘出一些信息：
      authorid：挖掘此信息主要是挖掘出用户对于author的偏好，主要可以挖掘的包括：
      ·观看历史中，authorid出现的次数；
      ·观看历史中，authorid所占的比例；
      ·观看历史中，authorid出现的次数占author作者所有作品的比例
      ·用户所有完整播放甚至重复播放的视频中，authorid出现的次数；
      ·用户所有完整播放甚至重复播放的视频中，authorid所占的比例；
      ·用户所有完整播放甚至重复播放的视频中，authorid占authorid出现次数的比例；
      ·用户停留时间很长的视频中，authorid出现的次数；
      ·用户停留时间很长的视频中，authorid所占的比例；
      ·用户停留时间很长的视频中，authorid占authorid出现次数的比例
      ·某一具体的历史action中，authorid出现的次数；
      ·某一具体的历史action中，authorid所占的比例；
      ·某一具体的历史action中，authorid占authorid出现次数的比例
      ·对于和某一action强相关的特征，也按照上面3个进行统计；
        - click_avatar 统计 follow 
        - like 统计 follow、comment
        - read_comment 统计 favorite、comment
      以上挖掘出的authorid特征都需要和视频的authorid做交叉，也就是只需要视频authorid对应的值就行了；
      bgm_song_id：挖掘此信息是为了挖掘出用户对于 bgm_song 的偏好，挖掘过程和authorid的挖掘过程相似；
      bgm_singer_id：挖掘此信息是为了挖掘出用户对于 bgm_singer 的偏好，挖掘过程和authorid的挖掘过程相似；
      videoplayseconds：视频时长信息，挖掘此信息是为了挖掘出用户对于视频市场的偏好信息；
      ·用户观看历史中，视频时间长度的均值、中位数、分位数均值（前25% 到 50%的均值）
      ·用户历史action中，视频时长的均值、中位数、分位数均值（前25% 到 50%的均值）
      ·和某一action强相关的action中，视频时长的均值、中位数、分位数均值（前25% 到 50%的均值）。
      ·用户所有完整播放的视频中，视频时长的均值、中位数、分位数均值（前25% 到 50%的均值）
      ·用户所有停留时间很长的视频中，视频时长的均值、中位数、分位数均值（前25% 到 50%的均值）
      上述特征的值均可以和视频时长做交叉，具体思路是，上面得到的各个值和视频时长之间的差距，为了衡量，可以采用 1/exp（|得到的值-视频时长|/视频时长）) 表示相关度、之后可以分别将多个相关度相乘做交叉；最终衡量时间上的用户偏好；
      top5_keywords和top5_keywords_weights: 挖掘此信息是为了挖掘用户对于关键词的偏好程度，也就是可以挖掘用户对于什么样关键词的视频更喜欢；
      ·用户历史观看视频中，keywords出现的比例和的权重，由各个视频的keywords相加的到；
      ·用户历史action视频中，keywords出现的比例和权重，有各个视频的keywords相加得到；
      ·和某一action强相关的action中，keywords出现的比例和权重，由各个视频的keywords相加得到；
      ·用户历史完整播放的action中，keywords出现的比例和权重，由各个视频的keywords相加得到；
      ·用户历史停留时间长的视频中，keywords出现的比例和权重，由各个视频的keywords统计得到；
      对于上述得到的用户兴趣，需要和视频做交叉，也就是只取视频的keywords对应出现的比例相加以及将视频keywords权重和用户兴趣权重相乘
      再可以将所有的相关权重或者出现次数融合成一个关键词的总特征；
      tag_list和tag_weights：挖掘此信息是为了挖掘用户对于视频标签的偏好，也就是挖掘用户喜欢什么样的视频；挖掘方式和前面关键词的挖掘类似即可；最终也需要将其和具体视频的特征做交叉；
      feed_embedding挖掘：挖掘此信息是为了挖掘用户对于和给出视频相似的视频的偏好信息；
      ·用户历史观看的视频embedding均值
      ·用户历史action的视频embedding均值；
      ·用户历史和某一action强相关的action的视频embedding均值
      ·用户完整播放的视频embedding均值
      ·用户停留时间长的embedding均值
      上面得到的embedding都需要再和对应视频的embedding算欧氏距离或者是余弦相似度；
      所有的欧氏距离或者余弦相似度可以再一次计算出feed_embedding相似度的总信息；
      
    2.过去7天内，视频的特征，这是和具体的用户无关的，而只是视频的特征信息，最后或者作为特征直接输入，或者和用户兴趣做一些交叉；
      feedid：挖掘这个特征主要是挖掘某个视频在过去7天之内流行度信息等等；
      ·过去7天之内，该视频出现的次数；
      ·过去7天之内，该视频某一action的次数以及占其出现次数的比例；
      ·过去7天之内，该视频在和某一action相关性最大的action次数以及占其出现次数的比例；
      ·过去7天之内，该视频被完整播放的次数以及占其出现次数的比例；
      authorid：挖掘这个特征的主要目的是挖掘视频作者的受欢迎程度；
      ·过去7天之内，该视频作者的作品出现的次数；
      ·过去7天之内，该视频作者的作品被action的次数，以及被action所占的比例；
      ·过去7天之内，该视频作者的作品被完整播放的次数，以及被完整播放的比例；
      bgm_song_id：挖掘该特征的目的是挖掘视频bgm的受欢迎程度等等，和authorid的处理比较类似；
      ·过去7天之内，该bgm_song_id出现的次数；
      ·过去7天之内，该bgm_song_id的作品被action的次数，以及被action所占的比例；
      ·过去7天之内，该bgm_song_id的视频被完整播放的次数，以及完整播放所占的比例；
      bgm_singer_id：挖掘该特征的目的是挖掘 bgm_singer 的受欢迎程度；
      ·过去7天之内，该bgm_singer_id出现的次数；
      ·过去7天之内，该bgm_singer_id的作品被action的次数，以及被action所占的比例；
      ·过去7天之内，该bgm_singer_id的视频被完整播放的次数以及完整播放所占的比例；

    3.其他信息的一些挖掘；
      用户的历史行为习惯，主要是用户的历史行为action占用户所有用户观看历史的比例、以及用户完整播放视频所占的比例等；
      视频发布时间信息，默认视频第一次出现的时间是其发布时间，可以加上视频发布时间距离最近的时间；
      device信息的挖掘，包括不同的action中，各种device所占的比例，或者不同device的action所占action的总数比例；以及用户不同的action中，device所占的比例等；

    4.在之前的特征基础上所做的交叉等等；


In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
import os
from scipy.spatial import distance
import pickle

In [2]:
user_action_path = "./wechat_algo_data1/user_action.csv"
feed_info_modified_path = "./feed_info_modified2.pkl"
test_data_path = "./wechat_algo_data1/test_a.csv"
user_interest_path = "./user_interest.pkl"

In [3]:
LABEL_COLUMNS = ['click_avatar', 'forward', 'follow', 'favorite', 'read_comment', 'comment', 'like']
ACTION_LIST = ['click_avatar', 'forward', 'follow', 'favorite', 'read_comment', 'comment', 'like', 'is_stay', 'is_finished', 'feedback', 'interested']

In [4]:
with open(feed_info_modified_path, 'rb') as file:
	feed_info_modified_df = pickle.load(file)
 
with open(user_interest_path, 'rb') as file:
  user_interest_df = pickle.load(file)

In [5]:
# 读取用户历史行为数据
user_action_df = pd.read_csv(user_action_path)
# 读取测试集数据
test_data_df = pd.read_csv(test_data_path)

In [6]:
#统计视频第一次出现的日期，作为视频的发布日期
feed_release_date = user_action_df.groupby(['feedid'])['date_'].apply(lambda x: x.min())

In [7]:
feed_release_date=pd.DataFrame(feed_release_date)
feed_info_modified_df = pd.merge(feed_info_modified_df, feed_release_date, how='left', on='feedid')

In [8]:
#将没有在之前出现过的视频发布日期设置为15
feed_info_modified_df['date_'].fillna(15, inplace=True)
feed_info_modified_df.rename(columns={'date_': 'release_date_'}, inplace=True)
print(feed_info_modified_df.columns)

Index(['feedid', 'authorid', 'videoplayseconds', 'bgm_song_id',
       'bgm_singer_id', 'author_n_feeds', 'videolength_bucket',
       'feed_embedding', 'tag_list', 'keyword_list', 'embedding_0',
       'embedding_1', 'embedding_2', 'embedding_3', 'embedding_4',
       'embedding_5', 'embedding_6', 'embedding_7', 'embedding_8',
       'embedding_9', 'embedding_10', 'embedding_11', 'embedding_12',
       'embedding_13', 'embedding_14', 'embedding_15', 'embedding_16',
       'embedding_17', 'embedding_18', 'embedding_19', 'embedding_20',
       'embedding_21', 'embedding_22', 'embedding_23', 'embedding_24',
       'embedding_25', 'embedding_26', 'embedding_27', 'embedding_28',
       'embedding_29', 'embedding_30', 'embedding_31', 'author_n_feeds_bucket',
       'release_date_'],
      dtype='object')


In [9]:
feed_info_df = feed_info_modified_df.copy()
feed_info_df.set_index(['feedid'], inplace=True)

In [10]:
#统一play和stay的单位：
user_action_df['play'] = user_action_df['play'] / 1000.0
user_action_df['stay'] = user_action_df['stay'] / 1000.0

In [11]:
#标记用户是否产生过反馈：
import numpy as np

user_action_df['feedback'] = (user_action_df[LABEL_COLUMNS].sum(axis=1) > 0).astype(np.int)
user_action_df['interested'] = (user_action_df[['favorite', 'read_comment', 'comment', 'like']].sum(axis=1) > 0).astype(np.int)

In [12]:
print(user_interest_df.columns)

Index(['userid', 'hist_tag', 'hist_keywords', 'hist_keywords_weights',
       'hist_tag_weights'],
      dtype='object')


In [13]:
user_interest_df.rename(columns={'stay': 'stay_sum'}, inplace=True)

In [14]:
user_hist_action_df = user_interest_df[['userid', 'watch_sum', 'watch_sum_bucket', 'stay_sum', 'stay_sum_bucket']]

KeyError: "['watch_sum' 'watch_sum_bucket' 'stay_sum' 'stay_sum_bucket'] not in index"

In [15]:
user_action_df = pd.merge(user_action_df, user_hist_action_df, on='userid', how='left')

NameError: name 'user_hist_action_df' is not defined

In [16]:
del user_hist_action_df

NameError: name 'user_hist_action_df' is not defined

In [17]:
user_interest_df.drop(columns=['watch_sum', 'watch_sum_bucket', 'stay_sum', 'stay_sum_bucket'], inplace=True)

KeyError: "labels ['watch_sum' 'watch_sum_bucket' 'stay_sum' 'stay_sum_bucket'] not contained in axis"

In [18]:
del user_interest_df

In [19]:
feed_info_modified_df.drop(columns=['bgm_song_id', 'bgm_singer_id'], inplace=True)

In [20]:
# 用户兴趣挖掘中三种 id 类特征挖掘可以采用完全相同的方式：
def User_Persona_ID(user_action_df, id, action_list):
  user_id_info = user_action_df.groupby(['userid', id]).size().reset_index()
  user_watch_sum = user_action_df.groupby('userid').size().reset_index()
  user_id_info = pd.merge(user_id_info, user_watch_sum, on='userid', how='left')
  user_id_info.rename(columns={'0_x': id + '_watched', '0_y': 'watched_sum'}, inplace=True)
  user_id_info[id + '_watched_rate'] = user_id_info[id + '_watched'] / user_id_info['watched_sum']
  for action in action_list:
    user_id_action = user_action_df[user_action_df[action] == 1].groupby(['userid', id]).size().reset_index()
    user_action_sum = user_action_df[user_action_df[action] == 1].groupby('userid').size().reset_index()
    user_id_action = pd.merge(user_id_action, user_action_sum, on='userid')
    user_id_action.rename(columns={'0_x': id + '_' + action, '0_y': id + '_' + action + '_sum'}, inplace=True)
    user_id_action[id + '_' + action + '_rate'] = user_id_action[id + '_' + action] / user_id_action[id + '_' + action + '_sum']
    user_id_info = pd.merge(user_id_info, user_id_action, on=['userid', id], how='left')
    user_id_info[id + '_' + action + '_partition'] = user_id_info[id + '_' + action] / user_id_info[id + '_watched']
  return user_id_info

In [21]:
#对用户历史观看视频的长度进行挖掘，分别挖掘出：用户观看过、用户有过action反馈的视频时间长短的均值和中位数
def User_Persona_Videoplayseconds(user_action_df, action_list):
  user_videoplayseconds_info = user_action_df.groupby('userid')['videoplayseconds'].mean().reset_index()
  user_videoplayseconds_info.rename(columns={'videoplayseconds': 'watch_videoplayseconds_mean'}, inplace=True)
  user_videoplayseconds_info = pd.merge(user_videoplayseconds_info, 
                      user_action_df.groupby('userid')['videoplayseconds'].median().reset_index(), 
                      on='userid', 
                      how='left')
  user_videoplayseconds_info.rename(columns={'videoplayseconds': 'watch_videoplayseconds_median'}, inplace=True)
  for action in action_list:
    user_videoplayseconds_info = pd.merge(user_videoplayseconds_info, 
                      user_action_df[user_action_df[action] == 1].groupby('userid')['videoplayseconds'].mean().reset_index(), 
                      on='userid', 
                      how='left')
    user_videoplayseconds_info.rename(columns={'videoplayseconds': action + '_videoplayseconds_mean'}, inplace=True)
    user_videoplayseconds_info = pd.merge(user_videoplayseconds_info, 
                      user_action_df[user_action_df[action] == 1].groupby('userid')['videoplayseconds'].median().reset_index(), 
                      on='userid', 
                      how='left')
    user_videoplayseconds_info.rename(columns={'videoplayseconds': action + '_videoplayseconds_median'}, inplace=True)
  return user_videoplayseconds_info

In [22]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

def create_feed_keyword_profile(user_hist_info):
  dataset = user_hist_info['keyword_list'].values
  from gensim.corpora import Dictionary
  dct = Dictionary(dataset)
  corpus = [dct.doc2bow(line) for line in dataset]
  model = TfidfModel(corpus)
  _feed_keywords = []
  _feed_keywords_weights = []
  for i in range(len(corpus)):
    vector = model[corpus[i]]
    feed_keywords = sorted(vector, key=lambda x: x[1], reverse=True)
    keywords_weights = dict(map(lambda x: (dct[x[0]], x[1]), feed_keywords))
    keywords = [i[0] for i in keywords_weights.items()]
    _feed_keywords.append(keywords)
    _feed_keywords_weights.append(keywords_weights)
  return _feed_keywords, _feed_keywords_weights

D:\anacanda\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [23]:
# 分别计算feed的各个tag_list的权重
def create_feed_tags_profile(user_hist_info):
  dataset = user_hist_info['tag_list'].values
  from gensim.corpora import Dictionary
  dct = Dictionary(dataset)
  corpus = [dct.doc2bow(line) for line in dataset]
  model = TfidfModel(corpus)
  _tags = []
  _tags_weights = []
  for i in range(len(corpus)):
    vector = model[corpus[i]]
    feed_tags = sorted(vector, key=lambda x: x[1], reverse=True)
    tags_weights = dict(map(lambda x: (dct[x[0]], x[1]), feed_tags))
    tags = [i[0] for i in tags_weights.items()]
    _tags.append(tags)
    _tags_weights.append(tags_weights)
  return _tags, _tags_weights

In [24]:
unique_user_df = pd.DataFrame(user_action_df['userid'].drop_duplicates())

In [25]:
def User_Hist_Interest(split_user_action_df):
  user_interested_df = split_user_action_df[split_user_action_df['feedback'] > 0].groupby('userid')[['keyword_list', 'tag_list']].sum().reset_index()
  _feed_keywords, _feed_keywords_weights = create_feed_keyword_profile(user_interested_df)
  user_interested_df['hist_keywords'] = _feed_keywords
  user_interested_df['hist_keywords_weights'] = _feed_keywords_weights
  tags, tags_weights = create_feed_tags_profile(user_interested_df)
  user_interested_df['hist_tag'] = tags
  user_interested_df['hist_tag_weights'] = tags_weights
  user_interested_df.drop(columns=['keyword_list', 'tag_list'], inplace=True)
  user_interested_df = pd.merge(unique_user_df, user_interested_df, on='userid', how='left')
  user_interested_df['hist_tag'] = user_interested_df['hist_tag'].apply(lambda x: x if type(x) != float else [])
  user_interested_df['hist_keywords'] = user_interested_df['hist_keywords'].apply(lambda x: x if type(x) != float else [])
  user_interested_df['hist_keywords_weights'] = user_interested_df['hist_keywords_weights'].apply(lambda x: x if type(x) != float else {})
  user_interested_df['hist_tag_weights'] = user_interested_df['hist_tag_weights'].apply(lambda x: x if type(x) != float else {})
  return user_interested_df

In [26]:
#挖掘用户历史的 embedding 信息
def User_Persona_Embedding(user_action_df, action_list):
  def feed_embedding_mean(feedid):
    return feed_info_df.loc[feedid]['feed_embedding'].mean()
  user_embedding_df = user_action_df.groupby('userid')['feedid'].unique().reset_index()
  user_embedding_df['feed_embedding'] = user_embedding_df['feedid'].apply(lambda x: feed_embedding_mean(x))
  user_embedding_df.rename(columns={'feed_embedding': 'watch_embedding'}, inplace=True)
  for action in action_list:
    user_action_embedding_df = user_action_df[user_action_df[action] == 1].groupby('userid')['feedid'].unique().reset_index()
    user_action_embedding_df['feed_embedding'] = user_action_embedding_df['feedid'].apply(lambda x: feed_embedding_mean(x))
    user_action_embedding_df.drop(columns=['feedid'], inplace=True)
    user_embedding_df = pd.merge(user_embedding_df, user_action_embedding_df, on='userid', how='left')
    user_embedding_df.rename(columns={'feed_embedding': action + '_embedding'}, inplace=True)
  return user_embedding_df

In [27]:
#挖掘每个视频相关的id类特征的信息
def Feed_Persona_Id(user_action_df, id, action_list):
  id_info = user_action_df.groupby(id).size().reset_index()
  id_info.rename(columns={0: 'feed_' + id + '_occurs_times'}, inplace=True)
  for action in action_list:
    id_action_info = user_action_df[user_action_df[action] == 1].groupby(id).size().reset_index()
    id_action_info.rename(columns={0: 'feed_' + id + '_' + action + '_times'}, inplace=True)
    # print(id_action_info.head())
    id_info = pd.merge(id_info, id_action_info, on=id, how='left')
    id_info['feed_' + id + '_' + action + '_rate'] = id_info['feed_' + id + '_' + action + '_times'] / id_info['feed_' + id + '_occurs_times']
  return id_info

In [28]:
# 用户行为习惯的挖掘
def User_Habit(user_action_df, action_list):
  user_habit_info = user_action_df.groupby('userid').size().reset_index()
  user_habit_info.rename(columns={0: 'past_watch_sum'}, inplace=True)
  for action in action_list:
    user_action = user_action_df[user_action_df[action] == 1].groupby('userid').size().reset_index()
    user_action.rename(columns={0: action + '_sum'}, inplace=True)
    user_habit_info = pd.merge(user_habit_info, user_action, on='userid', how='left')
    user_habit_info[action + '_rate'] = user_habit_info[action + '_sum'] / user_habit_info['past_watch_sum']
  return user_habit_info

In [29]:
#device信息的挖掘
def Device_Info(user_action_df, action_list):
  device_info = user_action_df.groupby('device').size().reset_index()
  device_info.rename(columns={0: 'device_sum'}, inplace=True)
  for action in action_list:
    device_action = user_action_df[user_action_df[action] == 1].groupby('device').size().reset_index()
    device_action.rename(columns={0: 'device_' + action + '_sum'}, inplace=True)
    device_info = pd.merge(device_info, device_action, on='device', how='left')
    device_info['device_' + action + '_rate'] = device_info['device_' + action + '_sum'] / device_info['device_sum']
  return device_info

In [30]:
#处理用户历史行为数据：
def ProcessingUserAction(user_action_test, feed_info_modified_df):
  user_action_test = pd.merge(user_action_test, feed_info_modified_df, on='feedid')
  user_action_test['is_finished'] = (user_action_test['play'] >= user_action_test['videoplayseconds']).astype(np.int)
  user_action_test['is_stay'] = (user_action_test['stay'] >= user_action_test['videoplayseconds'] * 2.0).astype(np.int)
  user_author_info = User_Persona_ID(user_action_test, 'authorid', ACTION_LIST)
  user_feed_info = user_action_test.groupby(['userid', 'feedid']).size().reset_index()
  user_feed_info.rename(columns={0: 'user_feedid_occurs'}, inplace=True)
  user_videoplayseconds_info = User_Persona_Videoplayseconds(user_action_test, ACTION_LIST)
  user_embedding_info = User_Persona_Embedding(user_action_test, ACTION_LIST)
  user_hist_interest_info = User_Hist_Interest(user_action_test)
  user_embedding_info.drop(columns=['feedid'], inplace=True)
  feedid_info = Feed_Persona_Id(user_action_test, 'feedid', ACTION_LIST)
  feed_author_info = Feed_Persona_Id(user_action_test, 'authorid', ACTION_LIST)
  user_habit_info = User_Habit(user_action_test, ACTION_LIST)
  device_info = Device_Info(user_action_test, ACTION_LIST)
  return user_feed_info, user_author_info, user_videoplayseconds_info, user_embedding_info, user_hist_interest_info, feedid_info, feed_author_info, user_habit_info, device_info

In [31]:
def keyword_tag_coocurrence(past_hist_list):
  coocurrence = []
  for past, hist in past_hist_list:
    cor = 0
    for word in past:
      if word in hist:
        cor += 1
    coocurrence.append(cor)
  return coocurrence


def keyword_tag_weights(past_list_hist_weight):
  coocurrence_weights = []
  for past, hist_weights in past_list_hist_weight:
    weight = 0
    for word in past:
      weight += hist_weights.get(word, 0)
    coocurrence_weights.append(weight)
  return coocurrence_weights 

In [32]:
#挖掘数据历史行为信息并将其作为下一天的特征
def DataProcessing(user_hist_actions, user_tag_test, feed_info_modified, date):
  user_feed_info, user_author_info, user_videoplayseconds_info, user_embedding_info, user_hist_interest_info, feedid_info, feed_author_info, user_habit_info, device_info = ProcessingUserAction(user_hist_actions, feed_info_modified)
  user_tag_test['date_'] = date
  user_tag_test = pd.merge(user_tag_test, feed_info_modified, on='feedid', how='left')
  # 将 user_feed_info 拼接
  user_tag_test = pd.merge(user_tag_test, user_feed_info, on=['userid', 'feedid'], how='left')
  user_tag_test['user_feedid_occurs'] = user_tag_test['user_feedid_occurs'].fillna(0)
  # 将 user_author_info 拼接，并将缺失值置为0
  user_tag_test = pd.merge(user_tag_test, user_author_info, on=['userid', 'authorid'], how='left')
  user_tag_test.drop(columns=['watched_sum'], inplace=True)
  columns = list(user_author_info.columns.values)
  for column in ['userid', 'authorid', 'watched_sum']:
    columns.remove(column)
  user_tag_test[columns] = user_tag_test[columns].fillna(0)
  # 将 user_videoplayseconds_info 拼接，并转化为计算和视频长度的差距
  user_tag_test = pd.merge(user_tag_test, user_videoplayseconds_info, on='userid', how='left')
  columns = list(user_videoplayseconds_info.columns.values)
  columns.remove('userid')
  for column in columns:
    user_tag_test[column] = 1.0 / np.exp(abs(user_tag_test[column] - user_tag_test['videoplayseconds']) / user_tag_test['videoplayseconds'])
  # 计算用户历史关键词和视频关键词之间的关系
  user_tag_test = pd.merge(user_tag_test, user_hist_interest_info, on='userid', how='left')
  user_tag_test['tag_cooccurrence'] = keyword_tag_coocurrence(user_tag_test[['tag_list', 'hist_tag']].values)
  user_tag_test['tag_cooccurrence_weights'] = keyword_tag_weights(user_tag_test[['tag_list', 'hist_tag_weights']].values)
  user_tag_test['keyword_cooccurrence'] = keyword_tag_coocurrence(user_tag_test[['keyword_list', 'hist_keywords']].values)
  user_tag_test['keyword_cooccurrence_weights'] = keyword_tag_weights(user_tag_test[['keyword_list', 'hist_keywords_weights']].values)
  # 视频特征和其他特征集中处理：
  other_id_info = [feedid_info, feed_author_info, user_habit_info, device_info]
  columns_to_drop = ['feedid', 'authorid', 'userid', 'device']
  for index, (id_info, column) in enumerate(zip(other_id_info, columns_to_drop)):
    user_tag_test = pd.merge(user_tag_test, id_info, on=column, how='left')
    columns = list(id_info.columns.values)
    columns.remove(column)
    user_tag_test[columns] = user_tag_test[columns].fillna(0)
  # 处理 embedding 特征，计算相似度
  def embedding_similarity(x):
    """
    计算余弦相似度距离
    """
    result = []
    for pair in x:
      result.append(distance.cosine(pair[0], pair[1]))
    return result

  def embedding_distance(x):
    """
    计算欧氏距离
    """
    result = []
    for pair in x:
      result.append(np.sqrt(np.sum((pair[0] - pair[1])**2)))
    return result
  
  user_tag_test = pd.merge(user_tag_test, user_embedding_info, on='userid', how='left')
  columns = list(user_embedding_info.columns.values)
  columns.remove('userid')
  for column in columns:
    user_tag_test[column + '_dist'] = embedding_distance(user_tag_test[[column, 'feed_embedding']].values) 
    user_tag_test[column + '_dist'] = user_tag_test[column + '_dist'].fillna(0)
    user_tag_test[column + '_sim'] = embedding_similarity(user_tag_test[[column, 'feed_embedding']].values)
    user_tag_test[column + '_sim'] = user_tag_test[column + '_sim'].fillna(0)
  # 视频热度信息（距离首次出现的时间）
  user_tag_test['date_gap'] = user_tag_test['date_'] - user_tag_test['release_date_']
  # 丢弃多余的行
  drop_columns1 = list(user_embedding_info.columns)
  drop_columns1.remove('userid')
  drop_columns2 = list(user_hist_interest_info.columns)
  drop_columns2.remove('userid')
  drop_columns = ['keyword_list', 'tag_list', 'feed_embedding', 'date_']
  drop_columns += drop_columns1
  drop_columns += drop_columns2
  user_tag_test.drop(columns=drop_columns, inplace=True)
  del user_feed_info, user_author_info, user_videoplayseconds_info, user_embedding_info, user_hist_interest_info, feedid_info, feed_author_info, user_habit_info, device_info
  return user_tag_test

In [33]:
user_test_action_df = user_action_df[(user_action_df['date_'] >= 7) & (user_action_df['date_'] <= 13)].copy()

In [34]:
user_tag_test_df = user_action_df[user_action_df['date_'] == 14][['userid', 'feedid', 'device', 'date_']].copy()

In [35]:
processed_data = DataProcessing(user_test_action_df, user_tag_test_df, feed_info_modified_df, 14)

D:\anacanda\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: overflow encountered in exp


In [53]:
def Split_and_Process(user_action_df, days): 
  """
  days: 滑动窗口的大小
  """
  for date in range(14 - days):
    user_action_train_split = user_action_df[(user_action_df['date_'] >= 1) & (user_action_df['date_'] <= date + days)]
    user_action_val_split = user_action_df[user_action_df['date_'] == date + days + 1][['userid', 'feedid', 'device'] + LABEL_COLUMNS]
    user_action_train_processed = DataProcessing(user_action_train_split, user_action_val_split, feed_info_modified_df, date + days + 1)
    user_action_train_processed.to_csv(os.path.join("./gbdt_processed_data", "user_action_" + str(date + days + 1) + '_version2.csv'))
    del user_action_train_split, user_action_val_split, user_action_train_processed
  user_action_train_split = user_action_df[(user_action_df['date_'] >= 1) & (user_action_df['date_'] <= 14)].copy()
  user_action_val_split = test_data_df
  user_action_train_processed = DataProcessing(user_action_train_split, user_action_val_split, feed_info_modified_df, 15)
  user_action_train_processed.to_csv(os.path.join("./gbdt_processed_data", "test_version2.csv"))

In [54]:
Split_and_Process(user_action_df, 7)

ValueError: can not merge DataFrame with instance of type <class 'pandas.core.series.Series'>

In [ ]:
processed_data.columns